### Imports

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from IPython.display import display
import json
import numpy as np
import pandas as pd
import os
import random
import re
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import tensorflow as tf

### Read scored test data

In [2]:
standard_data_path = 'gs://conversationai-models/biosbias/scored_data/test_standard_0409.csv'
scrubbed_data_path = 'gs://conversationai-models/biosbias/scored_data/test_scrubbed_0409.csv'
very_scrubbed_data_path = 'gs://conversationai-models/biosbias/scored_data/test_very_scrubbed_0409.csv'
gender_data_path = 'gs://conversationai-models/biosbias/scored_data/test_data_gender.csv'


perf_df = pd.read_csv(tf.gfile.Open(standard_data_path)).drop_duplicates(subset=['tokens'])
scrubbed_df = pd.read_csv(tf.gfile.Open(scrubbed_data_path)).drop_duplicates(subset=['tokens'])
very_scrubbed_df = pd.read_csv(tf.gfile.Open(very_scrubbed_data_path)).drop_duplicates(subset=['tokens'])
gender_df = pd.read_csv(tf.gfile.Open(gender_data_path)).drop_duplicates(subset=['tokens'])

In [3]:
print(perf_df.shape)
print(scrubbed_df.shape)

(59824, 300)
(59820, 36)


In [4]:
df = perf_df.join(scrubbed_df, rsuffix = '_scrubbed')
df = df.join(very_scrubbed_df, rsuffix = '_very_scrubbed')

In [5]:
df.head()

,tokens,gender,label,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_0,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_1,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_2,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_3,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_4,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_5,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_6,...,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_23,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_24,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_25,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_26,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_27,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_28,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_29,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_30,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_31,tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254_32
0,"[u'he', u'is', u'currently', u'working', u'clo...",M,25,0.000008,4.625991e-14,0.000089,0.000432,2.642943e-04,1.613340e-07,4.687537e-07,...,0.001929,1.914383e-06,0.000097,0.000332,7.086468e-07,8.798547e-16,0.000041,0.000395,0.000054,8.315536e-08
1,"[u'she', u'has', u'a', u'passion', u'for', u'w...",F,26,0.000001,5.970340e-18,0.000004,0.000155,8.439872e-06,1.380430e-07,8.653511e-09,...,0.013356,7.866625e-01,0.009269,0.024264,3.710595e-04,2.425320e-11,0.004488,0.002426,0.032467,1.274749e-04
2,"[u'growing', u'up', u'under', u'the', u'influe...",M,22,0.000205,1.023775e-15,0.008020,0.000054,1.159827e-06,2.420847e-06,4.043094e-06,...,0.000135,8.046401e-04,0.002173,0.000697,3.003297e-05,8.979249e-14,0.001901,0.000097,0.001727,4.318769e-06
3,"[u'he', u'earned', u'his', u'beng', u'degree',...",M,25,0.000009,1.354895e-13,0.001508,0.000051,1.071294e-07,1.333064e-08,1.857020e-05,...,0.009217,1.700057e-02,0.136035,0.009581,2.460610e-03,1.396903e-09,0.002276,0.009811,0.026841,1.840305e-04
4,"[u'her', u'professional', u'and', u'educationa...",F,25,0.001034,6.887217e-12,0.000701,0.021189,1.852501e-03,6.723991e-05,7.880444e-06,...,0.000425,9.174340e-08,0.995151,0.001635,9.952086e-11,4.422046e-14,0.000974,0.000039,0.000482,1.483144e-07


In [6]:
df.shape

(59824, 372)

In [7]:
df = df.dropna()
print(df.shape)

(59753, 372)


### Preprocessing

In [9]:
def get_class_from_col_name(col_name):
    #print(col_name)
    pattern = r'^.*_(\d+)$'
    return int(re.search(pattern, col_name).group(1))

In [10]:
def find_best_class(df, model_name, class_names):
    model_class_names = ['{}_{}'.format(model_name, class_name) for class_name in class_names]
    sub_df = df[model_class_names]
    df['{}_class'.format(model_name)] = sub_df.idxmax(axis=1).apply(get_class_from_col_name)

In [8]:
# Can check model names here
# df.columns.values

array(['tokens', 'gender', 'label',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_0',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_1',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_2',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_3',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_4',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_5',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_6',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_7',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_8',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_9',
       'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117_10',
       'tf_trainer_tf_gru_attention_mul

In [11]:
# May have to change.
# Can look them up in experiment tracker.
MODEL_NAMES = {
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_174837': 'debiased_tolga',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_174941': 'debiased_biosbias',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175003': 'strong_debiased_1',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175019': 'strong_debiased_2',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175034': 'strong_debiased_3',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175055': 'strong_debiased_4',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190328_103117': 'glove',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175113': 'strong_no_equalize',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175131': 'strong_no_projection', 
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190315_112954': 'scrubbed',
    'tf_trainer_tf_gru_attention_multiclass_biosbias_glove:v_20190410_175254': 'very_scrubbed'
}

In [12]:
CLASS_NAMES = range(33)

In [13]:
for _model in MODEL_NAMES:
    find_best_class(df, _model, CLASS_NAMES)

In [14]:
# Labels with either gender having too few examples
bad_labels = df.groupby('label').gender.value_counts().reset_index(name = 'count').query('count < 5').label.values
assert len(bad_labels) == 0

### Accuracy Calculation

In [15]:
accuracy_list = []
for _model in MODEL_NAMES:
    is_correct = (df['{}_class'.format(_model)] == df['label'])
    _acc = sum(is_correct)/len(is_correct)
    accuracy_list.append(_acc)
    print ('Accuracy for model {}: {}'.format(MODEL_NAMES[_model], _acc))

Accuracy for model debiased_biosbias: 0.806972034877
Accuracy for model very_scrubbed: 0.355915184175
Accuracy for model debiased_tolga: 0.818921225713
Accuracy for model strong_debiased_1: 0.817984034274
Accuracy for model strong_no_projection: 0.806687530333
Accuracy for model strong_debiased_2: 0.81733134738
Accuracy for model strong_no_equalize: 0.815239402206
Accuracy for model glove: 0.817950563152
Accuracy for model strong_debiased_4: 0.814737335364
Accuracy for model strong_debiased_3: 0.817599116362
Accuracy for model scrubbed: 0.130503907754


### Fairness Metrics

In [16]:
for _class in CLASS_NAMES:
    df['label_{}'.format(_class)] = (df['label'] == _class)

In [17]:
# Gender ratios of classes
gender_counts = df.groupby('label').gender.value_counts().reset_index(name = 'count')

In [18]:
def frac_female(df):
    m_count = df[df['gender'] == "M"]['count'].values[0]
    f_count = df[df['gender'] == "F"]['count'].values[0]
    return {'label': df['label'].values[0], 'frac_female': f_count/(m_count+f_count)}

In [19]:
frac_female_df = pd.DataFrame(list(gender_counts.groupby('label', as_index = False).apply(frac_female)))

In [20]:
def compute_tpr(df, _class, _model, threshold = 0.5):
    tpr = metrics.recall_score(df['label_{}'.format(_class)],
                               df['{}_{}'.format(_model,_class)] > threshold)
    return tpr
    
def compute_tpr_by_gender(df, _class, _model, threshold = 0.5):
    tpr_m = compute_tpr(df.query('gender == "M"'), _class, _model, threshold)
    tpr_f = compute_tpr(df.query('gender == "F"'), _class, _model, threshold)
    return {'M': tpr_m, 'F': tpr_f}

In [21]:
def compute_tpr_tnr(df, _class, _model, threshold = 0.5):
    #cm = metrics.confusion_matrix(df['label_{}'.format(_class)],
    #                              df['{}_{}'.format(_model,_class)] > threshold)
    cm = pd.crosstab(df['label_{}'.format(_class)], df['{}_{}'.format(_model,_class)] > threshold)
    #display(cm)
    if cm.shape[0] > 1:
        tn = cm.iloc[0,0]
        fp = cm.iloc[0,1]
        fn = cm.iloc[1,0]
        tp = cm.iloc[1,1]
        tpr = tp/(tp+fn)
        tnr = tn/(tn+fp)
    else:
        tpr = 0
        tnr = 1
    return tpr, tnr

def compute_tr_by_gender(df, _class, _model, threshold = 0.5):
    tpr_m, tnr_m = compute_tpr_tnr(df.query('gender == "M"'), _class, _model, threshold)
    tpr_f, tnr_f = compute_tpr_tnr(df.query('gender == "F"'), _class, _model, threshold)
    return {'TPR_m': tpr_m, 'TPR_f': tpr_f, 'TNR_m': tnr_m, 'TNR_f': tnr_f}

In [22]:
for _class in CLASS_NAMES:
    for _model in MODEL_NAMES:
        tpr_1 = compute_tpr(df, _class, _model)
        tpr_2, _ = compute_tpr_tnr(df, _class, _model)
        assert tpr_1 == tpr_2, '{} != {}'.format(tpr_1, tpr_2)
        #print('{} == {}'.format(tpr_1, tpr_2))

In [23]:
tpr_df = pd.DataFrame()
for _class in frac_female_df.label:
    row = {}
    row['label'] = _class
    for _model, _model_type in MODEL_NAMES.items():
        tpr, tnr = compute_tpr_tnr(df, _class, _model)
        row['{}_tpr'.format(_model_type)] = tpr
        row['{}_tnr'.format(_model_type)] = tnr
        gender_trs = compute_tr_by_gender(df, _class, _model)
        row['{}_tpr_F'.format(_model_type)] = gender_trs['TPR_f']
        row['{}_tpr_M'.format(_model_type)] = gender_trs['TPR_m']
        row['{}_tpr_gender_gap'.format(_model_type)] = gender_trs['TPR_f'] - gender_trs['TPR_m']
        row['{}_tnr_F'.format(_model_type)] = gender_trs['TNR_f']
        row['{}_tnr_M'.format(_model_type)] = gender_trs['TNR_m']
        row['{}_tnr_gender_gap'.format(_model_type)] = gender_trs['TNR_f'] - gender_trs['TNR_m']
    tpr_df = tpr_df.append(row, ignore_index = True)

In [24]:
results_df = pd.merge(tpr_df, frac_female_df, on = 'label')

In [25]:
TITLE_LABELS = [
    'accountant', 'acupuncturist', 'architect', 'attorney', 'chiropractor', 'comedian', 'composer', 'dentist',
    'dietitian', 'dj', 'filmmaker', 'interior_designer', 'journalist', 'landscape_architect', 'magician',
    'massage_therapist', 'model', 'nurse', 'painter', 'paralegal', 'pastor', 'personal_trainer',
    'photographer', 'physician', 'poet', 'professor', 'psychologist', 'rapper',
    'real_estate_broker', 'software_engineer', 'surgeon', 'teacher', 'yoga_teacher']

In [26]:
results_df['label_profession'] = results_df['label'].apply(lambda x: TITLE_LABELS[int(x)])

In [27]:
results_df[['frac_female']+['{}_tpr_gender_gap'.format(_model) for _model in MODEL_NAMES.values()]].corr()[['frac_female']]
    

,frac_female
frac_female,1.000000
debiased_biosbias_tpr_gender_gap,0.829982
very_scrubbed_tpr_gender_gap,0.458378
debiased_tolga_tpr_gender_gap,0.824882
strong_debiased_1_tpr_gender_gap,0.716922
strong_no_projection_tpr_gender_gap,0.709000
strong_debiased_2_tpr_gender_gap,0.596896
strong_no_equalize_tpr_gender_gap,0.772645
glove_tpr_gender_gap,0.794059
strong_debiased_4_tpr_gender_gap,0.550435


In [28]:
tpr_gender_gap_cols = ['{}_tpr_gender_gap'.format(_model) for _model in MODEL_NAMES.values()]
tnr_gender_gap_cols = ['{}_tnr_gender_gap'.format(_model) for _model in MODEL_NAMES.values()]

In [29]:
gender_gap_df = results_df[['label_profession', 'frac_female']+tpr_gender_gap_cols+tnr_gender_gap_cols]
#gender_gap_df.columns = ['label_profession', 'frac_female']+['{}'.format(_model) for _model in MODEL_NAMES.values()]

In [30]:
gender_gap_df.sort_values('frac_female', ascending = False)

,label_profession,frac_female,debiased_biosbias_tpr_gender_gap,very_scrubbed_tpr_gender_gap,debiased_tolga_tpr_gender_gap,strong_debiased_1_tpr_gender_gap,strong_no_projection_tpr_gender_gap,strong_debiased_2_tpr_gender_gap,strong_no_equalize_tpr_gender_gap,glove_tpr_gender_gap,...,very_scrubbed_tnr_gender_gap,debiased_tolga_tnr_gender_gap,strong_debiased_1_tnr_gender_gap,strong_no_projection_tnr_gender_gap,strong_debiased_2_tnr_gender_gap,strong_no_equalize_tnr_gender_gap,glove_tnr_gender_gap,strong_debiased_4_tnr_gender_gap,strong_debiased_3_tnr_gender_gap,scrubbed_tnr_gender_gap
7,dietitian,0.920437,0.290927,0.173878,0.297707,0.199900,0.223862,0.187072,0.250980,0.232835,...,-0.000674,-0.002692,-0.001220,-0.001617,-0.001530,-0.002213,-0.002810,-0.001630,-0.001412,-0.000266
13,nurse,0.914502,0.082735,0.013742,0.085377,0.048740,0.033271,0.025981,0.057404,0.082411,...,-0.001686,-0.007627,-0.007427,-0.004478,-0.004807,-0.004455,-0.005866,-0.002840,-0.002707,-0.001573
15,paralegal,0.866109,0.375755,0.094656,0.317482,0.262077,0.256944,0.271437,0.314915,0.271437,...,0.000247,-0.000103,0.000095,-0.000012,0.000075,-0.000219,-0.000164,-0.000060,-0.000236,0.000195
27,yoga_teacher,0.858696,0.276534,0.005518,0.143784,0.208049,0.116196,0.195067,0.161636,0.208374,...,0.000535,-0.001455,-0.001289,-0.000758,-0.001393,-0.001211,-0.001211,-0.001081,-0.001399,0.000299
12,model,0.818988,0.480652,0.176120,0.544309,0.418456,0.460211,0.455824,0.532551,0.505093,...,-0.001022,-0.000566,0.000379,0.000429,-0.000039,-0.000513,-0.001008,0.000249,0.000181,0.001161
10,interior_designer,0.782609,0.182716,-0.013580,0.243210,0.081481,0.096296,0.041975,0.224691,0.270370,...,0.000226,-0.000884,0.000032,-0.000023,0.000024,-0.000676,-0.000201,0.000216,-0.000013,0.000407
22,psychologist,0.620751,0.000799,0.008890,0.045876,0.043524,0.045169,0.020219,0.042056,0.017593,...,-0.000742,-0.005913,-0.004535,-0.002672,-0.002096,-0.004275,-0.002278,-0.003760,-0.002820,-0.001450
26,teacher,0.604382,0.111221,0.025352,0.129299,0.111760,0.113756,0.114246,0.119168,0.137121,...,-0.000813,-0.004694,-0.002497,-0.004570,-0.001141,-0.002609,-0.002664,-0.002461,-0.001785,0.000671
11,journalist,0.492152,0.019865,0.010182,0.057554,0.021920,0.001790,0.013070,0.042923,0.058686,...,0.000762,-0.000167,0.001286,0.001514,0.001955,0.000651,0.000014,0.001617,0.001571,-0.002623
19,physician,0.491203,0.019845,0.036850,0.056989,0.035120,0.042554,0.040719,0.034896,0.024797,...,0.001413,0.005790,0.006125,0.006385,0.006968,0.004761,0.007537,0.001844,0.004126,0.000307


In [31]:
# Fraction of comments where new model has lower
# TPR gap than the baseline

def compute_fraction_improved(df, baseline_model, improved_model):
    is_improved = np.abs(df[baseline_model]) >= np.abs(df[improved_model])
    return np.mean(is_improved)

In [32]:
for _model in MODEL_NAMES.values():
    print(_model)
    print(compute_fraction_improved(gender_gap_df, 'glove_tpr_gender_gap', '{}_tpr_gender_gap'.format(_model)))

debiased_biosbias
0.32142857142857145
very_scrubbed
0.7142857142857143
debiased_tolga
0.2857142857142857
strong_debiased_1
0.6428571428571429
strong_no_projection
0.6071428571428571
strong_debiased_2
0.7142857142857143
strong_no_equalize
0.39285714285714285
glove
1.0
strong_debiased_4
0.6071428571428571
strong_debiased_3
0.6071428571428571
scrubbed
0.8571428571428571


In [33]:
tpr_cols = ['{}_tpr_gender_gap'.format(_model) for _model in MODEL_NAMES.values()]
tnr_cols = ['{}_tnr_gender_gap'.format(_model) for _model in MODEL_NAMES.values()]
gender_gap_cols = tpr_cols + tnr_cols

In [34]:
gender_gap_df[gender_gap_cols].apply(lambda x: np.mean(x**2))

debiased_biosbias_tpr_gender_gap       0.029446
very_scrubbed_tpr_gender_gap           0.003786
debiased_tolga_tpr_gender_gap          0.028584
strong_debiased_1_tpr_gender_gap       0.014313
strong_no_projection_tpr_gender_gap    0.015602
strong_debiased_2_tpr_gender_gap       0.016134
strong_no_equalize_tpr_gender_gap      0.025152
glove_tpr_gender_gap                   0.022636
strong_debiased_4_tpr_gender_gap       0.016461
strong_debiased_3_tpr_gender_gap       0.014632
scrubbed_tpr_gender_gap                0.000189
debiased_biosbias_tnr_gender_gap       0.000011
very_scrubbed_tnr_gender_gap           0.000001
debiased_tolga_tnr_gender_gap          0.000009
strong_debiased_1_tnr_gender_gap       0.000009
strong_no_projection_tnr_gender_gap    0.000006
strong_debiased_2_tnr_gender_gap       0.000006
strong_no_equalize_tnr_gender_gap      0.000006
glove_tnr_gender_gap                   0.000008
strong_debiased_4_tnr_gender_gap       0.000003
strong_debiased_3_tnr_gender_gap       0

In [35]:
gender_gap_df[gender_gap_cols].apply(lambda x: np.mean(np.abs(x)))

debiased_biosbias_tpr_gender_gap       0.119049
very_scrubbed_tpr_gender_gap           0.041268
debiased_tolga_tpr_gender_gap          0.114932
strong_debiased_1_tpr_gender_gap       0.075670
strong_no_projection_tpr_gender_gap    0.079293
strong_debiased_2_tpr_gender_gap       0.075149
strong_no_equalize_tpr_gender_gap      0.102661
glove_tpr_gender_gap                   0.096764
strong_debiased_4_tpr_gender_gap       0.083171
strong_debiased_3_tpr_gender_gap       0.070882
scrubbed_tpr_gender_gap                0.007773
debiased_biosbias_tnr_gender_gap       0.002204
very_scrubbed_tnr_gender_gap           0.000958
debiased_tolga_tnr_gender_gap          0.002066
strong_debiased_1_tnr_gender_gap       0.001811
strong_no_projection_tnr_gender_gap    0.001657
strong_debiased_2_tnr_gender_gap       0.001537
strong_no_equalize_tnr_gender_gap      0.001866
glove_tnr_gender_gap                   0.001997
strong_debiased_4_tnr_gender_gap       0.001316
strong_debiased_3_tnr_gender_gap       0

In [36]:
def plot_tpr_gap(df, _model):
    fig, ax = plt.subplots(figsize=(15, 6))
    x = 'frac_female'
    y = '{}_tpr_gender_gap'.format(_model)
    p1 = sns.regplot(x = x, y = y, data = df)
    p1.set(xlabel = "% Female", ylabel = "TPR Gender Gap", title = _model)

    for line in range(0,df.shape[0]):
         p1.text(results_df[x][line]+0.01, df[y][line], df['label_profession'][line], horizontalalignment='left', size='medium', color='black')
    plt.show()

In [37]:
for _model in MODEL_NAMES.values():
    if 'untuned' in _model:
        plot_tpr_gap(results_df, _model)

In [38]:
results_df[['frac_female']+['{}_tpr_gender_gap'.format(_model) for _model in MODEL_NAMES.values()]].corr()[['frac_female']]

,frac_female
frac_female,1.000000
debiased_biosbias_tpr_gender_gap,0.829982
very_scrubbed_tpr_gender_gap,0.458378
debiased_tolga_tpr_gender_gap,0.824882
strong_debiased_1_tpr_gender_gap,0.716922
strong_no_projection_tpr_gender_gap,0.709000
strong_debiased_2_tpr_gender_gap,0.596896
strong_no_equalize_tpr_gender_gap,0.772645
glove_tpr_gender_gap,0.794059
strong_debiased_4_tpr_gender_gap,0.550435


### Gender Prediction Analysis

In [39]:
# Which model does this correspond to?
model_name = 'tf_gru_attention_multiclass_gender_biosbias_glove:v_20190405_142640'
gender_df['correct'] = ((gender_df['gender'] == 'M') == gender_df[model_name])

In [40]:
acc = gender_df.correct.sum()/gender_df.correct.count()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8423
